<a href="https://colab.research.google.com/github/theovincent/3DPointCloudClassification/blob/rangenet/range_net/RangeNet%2B%2B_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up

### Clone the RangeNet++ repository and install the dependencies

In [1]:
! git clone https://github.com/theovincent/lidar-bonnetal.git -b adapt

Cloning into 'lidar-bonnetal'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 167 (delta 46), reused 52 (delta 30), pack-reused 99
Receiving objects: 100% (167/167), 17.45 MiB | 10.64 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
import os 

os.chdir("/content/lidar-bonnetal/train/")

! pip install -r requirements.txt

os.chdir("/content/lidar-bonnetal/train/tasks/semantic")

     |████████████████████████████████| 676.9 MB 3.8 kB/s 
     |████████████████████▍           | 328.1 MB 1.4 MB/s eta 0:02:19

### Mount the drive

In [ ]:
from google.colab import drive

drive.mount('/content/lidar-bonnetal/train/tasks/semantic/mount')

Mounted at /content/lidar-bonnetal/train/tasks/semantic/mount


### Clone the 3DPointCloudClassification repository and install the dependencies

In [ ]:
os.chdir('/content/')
! git clone https://github.com/theovincent/3DPointCloudClassification.git -b rangenet

Cloning into '3DPointCloudClassification'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 248 (delta 130), reused 177 (delta 66), pack-reused 0
Receiving objects: 100% (248/248), 267.16 KiB | 10.69 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [ ]:
os.chdir('/content/3DPointCloudClassification')
! pip install -e .

Obtaining file:///content/3DPointCloudClassification
  Attempting uninstall: 3DPointCloudClassification
    Found existing installation: 3DPointCloudClassification 0.1
    Can't uninstall '3DPointCloudClassification'. No files were found to uninstall.
  Running setup.py develop for 3DPointCloudClassification


### Create the dataset

Get the files from Drive

In [ ]:
os.chdir("/content/lidar-bonnetal/train/tasks/semantic")

! cp -r mount/MyDrive/MVA/3DPointCloud/RangeNet++/data data

Create the dataset

In [ ]:
os.chdir("/content/lidar-bonnetal/train/tasks/semantic")
from range_net import CITY_INFERANCE_FOLDER

file = "MiniLille1"
folder = CITY_INFERANCE_FOLDER[file]

# !! CHANGE -itd OR 'nothing'
! create_dataset -f $file -itd -ns 1 -pts data_city/sequences/FOLDER/velodyne
! cp -r data_city/sequences/$folder data_city/sequences/08  # To fake a validation set
! cp -r data_city/sequences/$folder data_city/sequences/11  # To fake a test set
! create_dataset -f $file -itd -ns 10 -si -pts data_city/sequences/FOLDER/velodyne

{'file': 'MiniLille1', 'is_train_data': True, 'n_samples': 1, 'path_to_store': 'data_city/sequences/FOLDER/velodyne', 'store_indexes': False, 'store_ply': False}
100% 1/1 [00:00<00:00,  2.36it/s]
{'file': 'MiniLille1', 'is_train_data': True, 'n_samples': 10, 'path_to_store': 'data_city/sequences/FOLDER/velodyne', 'store_indexes': True, 'store_ply': False}
100% 10/10 [00:04<00:00,  2.35it/s]


Get the weights

In [ ]:
os.chdir("/content/lidar-bonnetal/train/tasks/semantic")

! cp -r mount/MyDrive/MVA/3DPointCloud/RangeNet++/model_1000 .

## Inference

In [ ]:
os.chdir('/content/3DPointCloudClassification')
! git pull
os.chdir('/content/lidar-bonnetal/train/tasks/semantic')
! git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/theovincent/3DPointCloudClassification
   1b59a8a..252abe1  rangenet   -> origin/rangenet
Updating 1b59a8a..252abe1
Fast-forward
 range_net/create_dataset.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
Already up to date.


In [ ]:
os.chdir('/content/lidar-bonnetal/train/tasks/semantic')

! ./infer.py --dataset data_city/ --log preds --model model_1000

----------
INTERFACE:
dataset data_city/
log preds
model model_1000
----------

Commit hash (training version):  b'756bf91'
----------

Opening arch config file from model_1000
Opening data config file from model_1000
train 00
train 01
train 02
train 03
train 04
train 05
train 06
train 07
train 09
train 10
valid 08
test 11
test 12
test 13
test 14
test 15
test 16
test 17
test 18
test 19
test 20
test 21
model folder exists! Using model from model_1000
Sequences folder exists! Using sequences from data_city/sequences
parsing seq 00
parsing seq 01
parsing seq 02
parsing seq 03
parsing seq 04
parsing seq 05
parsing seq 06
parsing seq 07
parsing seq 09
parsing seq 10
Using 10 scans from sequences [0, 1, 2, 3, 4, 5, 6, 7, 9, 10]
Sequences folder exists! Using sequences from data_city/sequences
parsing seq 08
Using 1 scans from sequences [8]
Sequences folder exists! Using sequences from data_city/sequences
parsing seq 11
parsing seq 12
parsing seq 13
parsing seq 14
parsing seq 15
parsing seq 1

## Merge labels

In [ ]:
! merge_labels -f $file -itd -pp preds/sequences/FOLDER/predictions

{'file': 'MiniLille1', 'is_train_data': True, 'path_predictions': 'preds/sequences/FOLDER/predictions'}
100% 3/3 [00:00<00:00,  4.16it/s]


### Save labels

In [ ]:
# !! CHANGE train OR test
! cp data/train/$file\_with_range_net_* mount/MyDrive/MVA/3DPointCloud/RangeNet++/data/train/

In [ ]:
! ls data/train/$file\_with_range_net_*

data/train/MiniLille1_with_range_net_3_samples.ply
